In [1]:
import src_lead_reporting as ld
import datascience as ds

In [2]:
lead=ld.lead_reporting_persona()

In [3]:
lead.create_table(start_year=2018)

Success
--------- Leads table created for Other for the year 2018 ---------
Success
--------- Leads table created for Other for the year 2019 ---------
Success
--------- Leads table created for Other for the year 2020 ---------
Success
--------- Leads table created for IT_Executive for the year 2018 ---------
Success
--------- Leads table created for IT_Executive for the year 2019 ---------
Success
--------- Leads table created for IT_Executive for the year 2020 ---------
Success
--------- Leads table created for Architect for the year 2018 ---------
Success
--------- Leads table created for Architect for the year 2019 ---------
Success
--------- Leads table created for Architect for the year 2020 ---------
Success
--------- Leads table created for Lead_Developer for the year 2018 ---------
Success
--------- Leads table created for Lead_Developer for the year 2019 ---------
Success
--------- Leads table created for Lead_Developer for the year 2020 ---------
Success
--------- Leads tabl

In [4]:
df=lead.calculations(2018)

--------- Lead Report for Other for the year 2018 ---------
Total number of leads: 16151
Marketing Accepted Leads: 6162
Sales Accepted Leads: 448
Sales Qualified Leads: 0
Total number of Converted Leads: 0
--------- Lead Report for Other for the year 2019 ---------
Total number of leads: 19999
Marketing Accepted Leads: 10898
Sales Accepted Leads: 569
Sales Qualified Leads: 1
Total number of Converted Leads: 1
--------- Lead Report for Other for the year 2020 ---------
Total number of leads: 13805
Marketing Accepted Leads: 5354
Sales Accepted Leads: 16
Sales Qualified Leads: 2
Total number of Converted Leads: 3
--------- Lead Report for IT_Executive for the year 2018 ---------
Total number of leads: 8496
Marketing Accepted Leads: 3182
Sales Accepted Leads: 146
Sales Qualified Leads: 0
Total number of Converted Leads: 1
--------- Lead Report for IT_Executive for the year 2019 ---------
Total number of leads: 12196
Marketing Accepted Leads: 7414
Sales Accepted Leads: 259
Sales Qualified L

In [5]:
df

,leads,Persona,Year,Stage
0,16151,Other,2018,Total Leads
0,6162,Other,2018,Marketing Accepted
0,448,Other,2018,Sales Accepted
0,0,Other,2018,Sales Qualified
0,0,Other,2018,Converted Leads
0,19999,Other,2019,Total Leads
0,10898,Other,2019,Marketing Accepted
0,569,Other,2019,Sales Accepted
0,1,Other,2019,Sales Qualified
0,1,Other,2019,Converted Leads


In [6]:
df.to_redshift('python_lead_analysis','na_field_marketing', if_exists='replace')

Full S3 url: rhdatasci/ops/python_lead_analysis.gzip
bucket where dataframe is to be copied: rhdatasci
keypath of file to be copied:ops/python_lead_analysis.gzip
Writing to disk...
Creating table na_field_marketing.python_lead_analysis
Querying Redshift...
drop table na_field_marketing.python_lead_analysis;

CREATE TABLE na_field_marketing.python_lead_analysis (
	leads BIGINT, 
	"Persona" TEXT, 
	"Year" BIGINT, 
	"Stage" TEXT
)

;
grant SELECT on na_field_marketing.python_lead_analysis
to group na_field_marketing_ro; commit;grant SELECT on na_field_marketing.python_lead_analysis
to group na_field_marketing_rw; commit;
bucket where file is to be removed: rhdatasci
keypath of file to be removed:ops/python_lead_analysis.gzip


### Creating other live dataset in rs

#### Step 1:

In [ ]:
query = '''

drop table na_field_marketing.temp_nafm_leads;
create table na_field_marketing.temp_nafm_leads as (  -- 493,680 rows
select   distinct lead_id , lead_date_created, l.converted_date,l.is_converted, p."eloqua persona", l.converted_opportunity_id, o.opportunity_forecast_category, o.opportunity_stage_name, o.opportunity_amount
from na_field_marketing.src_leads_global_sfdc l
join 
(
select distinct "lead contact id", p."eloqua persona"
from na_field_marketing.prod_nafm_mt p
where  "email (person)" != '%ibm%' 
) p
on left(l.lead_id,15) = "lead contact id"
left join na_field_marketing.src_opprtnty_sfdc o on l.converted_opportunity_id = o.opportunity_id
where lead_date_created >= '2018-01-01' 
);


'''

ds.query_RS(query, instance='DS', option='execute')

#### Step 2:

In [ ]:
query = '''

drop table na_field_marketing.temp_nafm_lmh;
create table na_field_marketing.temp_nafm_lmh as (
select  l.lead_stage_from,l.lead_stage_to ,t.*
from na_field_marketing.src_lmh_sfdc l
join na_field_marketing.temp_nafm_leads t
on l.lead_id = t.lead_id
);

'''

ds.query_RS(query, instance='DS', option='execute')